<a href="https://colab.research.google.com/github/damsdaddy/BDC101/blob/main/FRIENDS_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step1. 라이브러리를 로드합니다.

In [1]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

Step2. 구글 드라이브를 마운트합니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step3. 파일을 불러옵니다.

In [3]:
mypath = '/content/drive/MyDrive/Colab Notebooks/dataset/' # 구글 드라이브에 파일이 있는 폴더경로를 설정
train_data_json = pd.read_json(mypath+'friends_train.json', orient='records', typ='series')
dev_data_json = pd.read_json(mypath+'friends_dev.json', orient='records', typ='series')
test_data_json = pd.read_json(mypath+'friends_test.json', orient='records', typ='series')

Step4. 데이터가 정상적으로 로드되었는지를 확인합니다.

In [4]:
train_data_json[:10]

0    [{'speaker': 'Chandler', 'utterance': 'also I ...
1    [{'speaker': 'Chandler', 'utterance': 'Hey, Mo...
2    [{'speaker': 'Rachel', 'utterance': 'Hey!', 'e...
3    [{'speaker': 'Chandler', 'utterance': 'Good jo...
4    [{'speaker': 'Joey', 'utterance': 'Okay, look,...
5    [{'speaker': 'Chandler', 'utterance': 'What've...
6    [{'speaker': 'Phoebe', 'utterance': 'Nobody!',...
7    [{'speaker': 'Chandler', 'utterance': 'Hey!', ...
8    [{'speaker': 'Mona', 'utterance': 'I love your...
9    [{'speaker': 'Ross', 'utterance': 'Get Out!', ...
dtype: object

In [5]:
dev_data_json[:10]

0    [{'speaker': 'Phoebe', 'utterance': 'Oh my God...
1    [{'speaker': 'Joey', 'utterance': 'Hey Estelle...
2    [{'speaker': 'Ross', 'utterance': 'Yes? Yes?! ...
3    [{'speaker': 'Ross', 'utterance': 'You had no ...
4    [{'speaker': 'Phoebe', 'utterance': 'Oh, it's ...
5    [{'speaker': 'Monica', 'utterance': 'Hey! What...
6    [{'speaker': 'Joey', 'utterance': 'Hey, whats...
7    [{'speaker': 'Ross', 'utterance': 'So where ar...
8    [{'speaker': 'Monica', 'utterance': 'Well, you...
9    [{'speaker': 'Dr. Harad', 'utterance': 'Okay. ...
dtype: object

In [6]:
test_data_json[:10]

0    [{'speaker': 'Mark', 'utterance': 'Why do all ...
1    [{'speaker': 'Joey', 'utterance': 'Come on, Ly...
2    [{'speaker': 'Ross', 'utterance': 'Okay.', 'em...
3    [{'speaker': 'Phoebe', 'utterance': 'Ohh!', 'e...
4    [{'speaker': 'Joey', 'utterance': 'Previously ...
5    [{'speaker': 'Jade', 'utterance': 'Oh, Bob, he...
6    [{'speaker': 'Phoebe', 'utterance': 'Coming th...
7    [{'speaker': 'Rachel', 'utterance': 'Well Phoe...
8    [{'speaker': 'Ross', 'utterance': 'Whats all ...
9    [{'speaker': 'Roger', 'utterance': 'Oh.', 'emo...
dtype: object

Step5. 빈 데이터프레임을 생성합니다.

In [7]:
train_data = pd.DataFrame(columns = ['speaker' , 'utterance', 'emotion', 'annotation'])
test_data = pd.DataFrame(columns = ['speaker' , 'utterance', 'emotion', 'annotation'])

Step6. 데이터프레임에 json 데이터를 넣습니다.

In [8]:
for i in tqdm(train_data_json):
    for j in list(i):
        train_data = train_data.append(j, ignore_index=True)

In [9]:
for i in tqdm(dev_data_json):
    for j in list(i):
        train_data = train_data.append(j, ignore_index=True)

In [10]:
for i in tqdm(test_data_json):
    for j in list(i):
        test_data = test_data.append(j, ignore_index=True)

Step7. 데이터가 잘 들어갔는지 확인합니다.

In [11]:
train_data

,speaker,utterance,emotion,annotation
0,Chandler,also I was the point person on my companys tr...,neutral,4100000
1,The Interviewer,You mustve had your hands full.,neutral,5000000
2,Chandler,That I did. That I did.,neutral,5000000
3,The Interviewer,So lets talk a little bit about your duties.,neutral,5000000
4,Chandler,My duties? All right.,surprise,2000030
...,...,...,...,...
11734,Monica,No.,sadness,2030000
11735,Rachel,What? Oh my God! Im gonna miss you so much!,sadness,0040010
11736,Monica,Im gonna miss you!,sadness,0050000
11737,Rachel,I mean its the end of an era!,sadness,0050000


In [12]:
test_data

,speaker,utterance,emotion,annotation
0,Mark,Why do all youre coffee mugs have numbers on ...,surprise,2000030
1,Rachel,Oh. Thats so Monica can keep track. That way ...,non-neutral,2100011
2,Rachel,Y'know what?,neutral,3000020
3,Ross,It didnt.,neutral,5000000
4,Frank,"Okay, so what you used to have with Rachel, is...",joy,1300010
...,...,...,...,...
2759,Phoebe,"Hey, why dont you guys go get portraits done ...",neutral,5000000
2760,Monica,Thats a good idea! I bet they have one of tho...,joy,1400000
2761,Phoebe,"Yeah thats great! Next to that, Chandler won...",neutral,3100100
2762,Monica,Chandler what do you say?,neutral,5000000


Step8. 전처리를 수행합니다.
- 휴리스틱 방식으로 토큰화가 잘 이루어지지 않는 부분을 공백을 삽입합니다.
- 대문자를 소문자로 만듭니다.
- 중복 데이터를 제거합니다.
- 공백만 있는 경우 Null 로 치환하고, Null행은 제거합니다.
- 감정단어를 인덱스(정수값)로 변환합니다.

In [13]:
# train_data 전처리

# 휴리스틱 검출규칙 : 비표준어에 대해 직관적인 방법으로 규칙을 만들어 교체
heuristic_rule = [
    ['+', "'"],
    ['-', ' - '],
    ['  +', ' ']
]

#train_data['utterance'] = train_data['utterance'].str.replace("[^0-9a-zA-Z ]","") # 정규 표현식 수행
train_data['utterance'] = train_data['utterance'].str.lower()

for rule in tqdm(heuristic_rule):
    train_data['utterance'] = train_data['utterance'].str.replace(rule[0],rule[1]) # 휴리스틱 규칙적용


train_data.drop_duplicates(subset = ['utterance'], inplace=True) # 열에서 중복인 내용이 있다면 중복 제거
print('열 중복제거 후 테스트용 샘플의 개수 :',len(train_data))
train_data['utterance'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거
print('Null값 제거 후 테스트용 샘플의 개수 :',len(train_data))

# 감정단어를 숫자값으로 변환
change_value_dict = {'neutral':1, 'joy':2, 'sadness':3, 'fear':4, 'anger':5, 'surprise':6, 'disgust':7, 'non-neutral':8}
train_data = train_data.replace({'emotion':change_value_dict})



열 중복제거 후 테스트용 샘플의 개수 : 10268
Null값 제거 후 테스트용 샘플의 개수 : 10268


In [14]:
# test_data 전처리
#test_data['utterance'] = test_data['utterance'].str.replace("[^0-9a-zA-Z ]","") # 정규 표현식 수행
test_data['utterance'] = test_data['utterance'].str.lower()

for rule in tqdm(heuristic_rule):
    test_data['utterance'] = test_data['utterance'].str.replace(rule[0],rule[1]) # 휴리스틱 규칙적용

test_data.drop_duplicates(subset = ['utterance'], inplace=True) # 열에서 중복인 내용이 있다면 중복 제거
test_data['utterance'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

# 감정단어를 숫자값으로 변환
test_data = test_data.replace({'emotion':change_value_dict})

print('전처리 후 테스트용 샘플의 개수 :',len(test_data))


전처리 후 테스트용 샘플의 개수 : 2500


Step9. 정상처리되었는지 확인합니다.

In [15]:
train_data

,speaker,utterance,emotion,annotation
0,Chandler,also i was the point person on my company's tr...,1,4100000
1,The Interviewer,you must've had your hands full.,1,5000000
2,Chandler,that i did. that i did.,1,5000000
3,The Interviewer,so let's talk a little bit about your duties.,1,5000000
4,Chandler,my duties? all right.,6,2000030
...,...,...,...,...
11732,Monica,"yeah, sweetie.",1,3200000
11733,Rachel,"i mean we're not, we're not gonna live togethe...",3,0040010
11735,Rachel,what? oh my god! i'm gonna miss you so much!,3,0040010
11736,Monica,i'm gonna miss you!,3,0050000


In [16]:
test_data

,speaker,utterance,emotion,annotation
0,Mark,why do all you're coffee mugs have numbers on ...,6,2000030
1,Rachel,oh. that's so monica can keep track. that way ...,8,2100011
2,Rachel,y'know what?,1,3000020
3,Ross,it didn't.,1,5000000
4,Frank,"okay, so what you used to have with rachel, is...",2,1300010
...,...,...,...,...
2759,Phoebe,"hey, why don't you guys go get portraits done ...",1,5000000
2760,Monica,that's a good idea! i bet they have one of tho...,2,1400000
2761,Phoebe,"yeah that's great! next to that, chandler won'...",1,3100100
2762,Monica,chandler what do you say?,1,5000000


Step10. 전처리결과를 다음단계에서 사용할 수 있도록 csv 파일로 저장합니다.

In [17]:
train_data.to_csv(mypath+'friends_train_preprocess.csv',sep='|')
test_data.to_csv(mypath+'friends_test_preprocess.csv',sep='|')